## ANALISIS TRACK BASADO EN PWD

In [1]:
#PASO_1
#Importamos un xml procedente de pwd

from xml.dom.minidom import parse
import xml.dom.minidom
import pandas as pd

path='Mallorca70.3_tp'
DOMTree = xml.dom.minidom.parse(path+".xml")


collection = DOMTree.documentElement

trkpts = collection.getElementsByTagName("sample")
list_track = []

count=0
for trkpt in trkpts:
    count +=1
    if len(trkpt.getElementsByTagName("timeoffset"))==0:
        timeoffset = ''
    else:
        timeoffset = float(trkpt.getElementsByTagName("timeoffset")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("hr"))==0:
        hr = ''
    else:
        hr = float(trkpt.getElementsByTagName("hr")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("spd"))==0:
        spd = ''
    else:
        spd = float(trkpt.getElementsByTagName("spd")[0].firstChild.nodeValue)    
    
    if len(trkpt.getElementsByTagName("pwr"))==0:
        pwr = ''
    else:
        pwr = float(trkpt.getElementsByTagName("pwr")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("cad"))==0:
        cad = ''
    else:
        cad = float(trkpt.getElementsByTagName("cad")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("dist"))==0:
        dist = ''
    else:
        dist = float(trkpt.getElementsByTagName("dist")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("lat"))==0:
        lat = ''
    else:
        lat = float(trkpt.getElementsByTagName("lat")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("lon"))==0:
        lon = ''
    else:
        lon = float(trkpt.getElementsByTagName("lon")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("alt"))==0:
        alt = ''
    else:
        alt = float(trkpt.getElementsByTagName("alt")[0].firstChild.nodeValue)    

    if len(trkpt.getElementsByTagName("temp"))==0:
        temp = ''
    else:
        temp = float(trkpt.getElementsByTagName("temp")[0].firstChild.nodeValue)
   
    list_track.append([timeoffset, hr, spd, pwr, cad, dist, lat, lon, alt, temp])

print ("Puntos cargados: %d" %count)

    

result = pd.DataFrame(list_track)
result.columns=["timeoffset", "hr", "spd", "pwr", "cad", "dist", "lat", "lon", "alt", "temp"]
result['prueba'] = path
result.to_excel('Intermedio'+path+'.xlsx')
result.to_excel(path+".xlsx")
print('finish')



FileNotFoundError: [Errno 2] No such file or directory: 'Mallorca70.3_tp.xml'

## ANALISIS TRACK BASADO EN GPX

In [2]:
#PASO_1
#Importamos un xml procedente de gpx

from xml.dom.minidom import parse
import xml.dom.minidom
import pandas as pd


#path='Analisis_Aerodinamica'
#path='Entreno_preparatorio'
#path='2017_05_13_Triatlón'
path='20180406_LEVS_195_90_R_E'
DOMTree = xml.dom.minidom.parse(path+".xml")

collection = DOMTree.documentElement

trkpts = collection.getElementsByTagName("trkpt")
list_track = []

def to_dt(dato):
    t=pd.to_datetime(dato)
    return t.hour*3600+t.minute*60+t.second

count=0

for trkpt in trkpts:
    count +=1
    if trkpt.hasAttribute("lat"):
      lat = float(trkpt.getAttribute("lat"))
    if trkpt.hasAttribute("lon"):
      lon = float(trkpt.getAttribute("lon"))
    
    if len(trkpt.getElementsByTagName("ele"))==0:
        alt = ''
    else:
        alt = float(trkpt.getElementsByTagName("ele")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("time"))==0:
        time = ''
    else:
        time = trkpt.getElementsByTagName("time")[0].firstChild.nodeValue
    
    if len(trkpt.getElementsByTagName("gpxdata:hr"))==0:
        hr = ''
    else:
        hr = float(trkpt.getElementsByTagName("gpxdata:hr")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:cadence"))==0:
        cad = ''
    else:
        cad = float(trkpt.getElementsByTagName("gpxdata:cadence")[0].firstChild.nodeValue)
        
    if len(trkpt.getElementsByTagName("gpxdata:temp"))==0:
        temp = ''
    else:
        temp = float(trkpt.getElementsByTagName("gpxdata:temp")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:distance"))==0:
        dist = ''
    else:
        dist = float(trkpt.getElementsByTagName("gpxdata:distance")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:power"))==0:
        pwr = ''
    else:
        pwr = float(trkpt.getElementsByTagName("gpxdata:power")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("gpxdata:speed"))==0:
        spd = ''
    else:
        spd = float(trkpt.getElementsByTagName("gpxdata:speed")[0].firstChild.nodeValue)
    
    timeoffset = to_dt(time)
    
    list_track.append([timeoffset, hr, spd, pwr, cad, dist, lat, lon, alt, temp])

result = pd.DataFrame(list_track)
result.columns = ["timeoffset", "hr", "spd", "pwr", "cad", "dist", "lat", "lon", "alt", "temp"]
result['prueba'] = path
print ("Puntos cargados: %d" %count)
result.to_excel('Intermedio'+path+'.xlsx')
print('finish')


Puntos cargados: 7586
finish


In [ ]:
#conda install -c conda-forge geopy

## ENRIQUECEMOS EL FICHERO. COMUN PARA TODOS LOS ORIGENES

In [3]:
import pandas as pd
import math as mt
from scipy.signal import savgol_filter

def sync_bearing(ori_bearing):
    if ori_bearing < -90:
        new_bearing = 450+ori_bearing
    else:
        new_bearing = ori_bearing+90
    return new_bearing

#path='Analisis_Aerodinamica'
#path='Entreno_preparatorio'
#path='2017_05_13_Triatlón'
path='20180406_LEVS_195_90_R_E'

df = pd.read_excel('Intermedio'+path+'.xlsx')

df2 = df[df['lat'] >0]
df2['pwr'].fillna(0, inplace=True)
#df2.drop(['dist','cad','hr','spd'], axis=1, inplace=True)

from geopy.distance import distance
from geopy import Point
i=1
dist_c=[]
dist_c.append(0)
ele_gain = []
ele_gain.append(0)
d=0
e=0
porc=[]
porc.append(0)
spd_c=[]
spd_c.append(0)
spd_prev=[]
spd_prev.append(0)
ele_prev=[]
ele_prev.append(0)
bearing=[]
bearing.append(0)
timeoffset_prev=[]
timeoffset_prev.append(0)


while i < len(df2):
    #print("Punto %d" %i
    punto1 = Point(df2.iloc[i-1]['lon'], df2.iloc[i-1]['lat'])
    punto2 = Point(df2.iloc[i]['lon'], df2.iloc[i]['lat'])
    dist_c.append(distance(punto1, punto2).m)
    d = distance(punto1, punto2).m
    spd_c.append(float(d/(df2.iloc[i]['timeoffset']-df2.iloc[i-1]['timeoffset'])))
    punto1 = df2.iloc[i-1]['alt']
    punto2 = df2.iloc[i]['alt']
    ele_gain.append((punto2)-(punto1))
    e = punto2-punto1
    porc.append(e*100/d)
    lona = df2.iloc[i-1]['lon']
    lata = df2.iloc[i-1]['lat']
    lonb = df2.iloc[i]['lon']
    latb = df2.iloc[i]['lat']
    X = mt.cos(lonb)*mt.sin(latb-lata)
    Y = mt.cos(lona)*mt.sin(lonb)-mt.sin(lona)*mt.cos(lonb)*mt.cos(latb-lata)
    
    #bearing.append(mt.atan2(X,Y)*57.2957795)  Se modifica por la siguiente línea que corrige estandariza el ángulo
    bearing.append(sync_bearing(mt.atan2(X,Y)*57.2957795))
    
    i+=1
df2['dist_c']=dist_c
df2['ele_gain']=ele_gain
df2['porc']=porc
df2['spd_c']=spd_c
df2['bearing']=bearing
i=1
while i < len(df2):
    spd_prev.append(df2.iloc[i-1]['spd_c'])
    ele_prev.append(df2.iloc[i-1]['alt'])
    timeoffset_prev.append(df2.iloc[i-1]['timeoffset'])
    i+=1
df2['spd_prev']=spd_prev
df2['ele_prev']=ele_prev
df2['timeoffset_prev']=timeoffset_prev

#aplicamos el suavizado de señal
axis_y = df["alt"]
axis_yhat = savgol_filter(axis_y, 31, 2)
df2["new_ele"] = axis_yhat

tramos = [0]
tramo = 0
pendiente = 0
pendiente_ant = 0


lista_tramos = [(0,0,"Inicio")]

lista_tramos_2 = []
tramo_inicial = 0

i = 1
print("Entramos en el While1 %d" % len(df2.index))
while i < len(df2.index):
    #print("punto = %d y tramo = %d" % (i, tramo))
    pendiente = df2.iloc[i]["new_ele"]-df2.iloc[i-1]["new_ele"]

    if pendiente * pendiente_ant >=0:
        tramos.append(tramo)
    else:
        lista_tramos_2.append((tramo, tramo_inicial, i-1))
        tramo_inicial = i
        lista_tramos.append((tramo,i-1,"Fin"))
        tramo +=1
        tramos.append(tramo)
        lista_tramos.append((tramo,i,"Inicio"))
    i+=1
    pendiente_ant = pendiente

print("tramos : %d" % len(tramos))
print("Salimos del While")
df2["tramo"] = tramos






#Calculo de tramos de 100m

tramos_100 = [0]
tramo = 0
acum_dist = 0
i = 1

print("Entramos en el While2 %d" % len(df2.index))

df2.iloc[0,5] = 0

while i < len(df2.index):
   
    acum_dist = df2.iloc[i-1]["dist"]
    tramo = mt.floor(acum_dist/100)
    tramos_100.append(tramo)
    i +=1
print("tramos_100m : %d" % len(tramos_100))
print("Salimos del While2")
df2["tramos_100"] = tramos_100






df2.to_excel(path+'_ml.xlsx')

print('finish')


C:\Users\eduardo.hernanz\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in true_divide


Entramos en el While1 7586
tramos : 7586
Salimos del While
Entramos en el While2 7586
tramos_100m : 7586
Salimos del While2
finish


In [4]:
lista_tramos_2

[(0, 0, 57),
 (1, 58, 64),
 (2, 65, 68),
 (3, 69, 71),
 (4, 72, 72),
 (5, 73, 101),
 (6, 102, 105),
 (7, 106, 431),
 (8, 432, 464),
 (9, 465, 467),
 (10, 468, 468),
 (11, 469, 470),
 (12, 471, 472),
 (13, 473, 527),
 (14, 528, 553),
 (15, 554, 593),
 (16, 594, 600),
 (17, 601, 637),
 (18, 638, 690),
 (19, 691, 706),
 (20, 707, 708),
 (21, 709, 710),
 (22, 711, 711),
 (23, 712, 748),
 (24, 749, 749),
 (25, 750, 895),
 (26, 896, 986),
 (27, 987, 1027),
 (28, 1028, 1091),
 (29, 1092, 1125),
 (30, 1126, 1142),
 (31, 1143, 1152),
 (32, 1153, 1153),
 (33, 1154, 1203),
 (34, 1204, 1252),
 (35, 1253, 1288),
 (36, 1289, 1289),
 (37, 1290, 1290),
 (38, 1291, 1292),
 (39, 1293, 1294),
 (40, 1295, 1306),
 (41, 1307, 1374),
 (42, 1375, 1375),
 (43, 1376, 1756),
 (44, 1757, 1758),
 (45, 1759, 1919),
 (46, 1920, 1923),
 (47, 1924, 2001),
 (48, 2002, 2018),
 (49, 2019, 2019),
 (50, 2020, 2020),
 (51, 2021, 2813),
 (52, 2814, 3399),
 (53, 3400, 3460),
 (54, 3461, 3482),
 (55, 3483, 3521),
 (56, 3522, 3